In [5]:
#利用多元线性回归，预测股票收盘价格

import time

def main_data_linear():
    d_time = time.time() #计算程序运行耗时
    
    #如果希望计算最新的日期，定义获取数据的起始日期
    end_date = datetime.datetime.now()
    start_date = end_date - datetime.timedelta(days = 1000) #模型采用1000个自然天数据
    end_date = end_date.strftime('%Y%m%d')
    start_date = start_date.strftime('%Y%m%d')
    
    #如果希望获取之前的数据，在此处赋值截至日期。若否，将下两行屏蔽掉
    end_date = '20200107' #直接赋值日期
    start_date = str(int(end_date[0:4])-3) + end_date[4:8] #模型采用3年的数据

    #参数的赋值
    ty = 1 #0表示主板和中小板，1表示上交所主板，2表示深交所主板，3表示深交所中小板
    forecast_out = 2 # 定义预测天数

    #是否需要同时计算相关系数data_corr，0表示不需要，1表示需要
    with_corr = 0         

    
    #获取股票代码
    stock_code = acq_code(ty) 
    
    if with_corr == 1:
        sts_data = [[]for i in range(len(stock_code))] #当需要计算corr时，这里定义corr为数组。
    
    result = [] #存放计算结果
        
    for i in range(len(stock_code)):
        
        #获取股票数据
        df = acq_data(stock_code[i], start_date, end_date) 
        
        #当需要计算corr时，这里获取计算数据
        if with_corr == 1:
            sts_data[i] = df.close #这里使用close字段作为计算相关系数的依据。也可以定义为其他字段，例如open。参考tushare中daily的参数。

        if len(df) > 100: #当数据量大于100时（也代表样本数大于100），才执行运算。
            
            #整理线性回归所需数据
            X, y, X_lately = general_data(df, stock_code[i], forecast_out) 
            
            if len(y) > 100: #当数据量大于100时（也代表样本数大于100），才执行运算。
                
                #进行线性回归计算
                income, accuracy, forecast_set = general_linear(X, y, X_lately) 
            
                if accuracy > 0.95: #当准确率大于95%时，才保存结果。
                    
                    #保存计算结果到数组
                    result = append_data(stock_code[i], income, accuracy, result) 
   
                    #绘制图形
                    #draw_plot(df, end_date, stock_code[i], forecast_set) 
        
        if int(i/100)==(i/100) and i>0: #进度提示
            print('已计算：',i,'只股票数据，继续计算……')

    print('符合要求的股票共有：',len(result), '只。')


    fn = 'data_linear.csv' #定义结果保存的的文件名称
    save_to_file(result, fn) #保存计算结果到文件

    print('完成！线性回归运行结果保存在文件：',fn)

    
    #计算相关系数
    if with_corr == 1:
        #计算相关系数，并保存到文件
        fn = cal_corr(stock_code,sts_data) 
        print('完成！相关系数结果保存为文件：',fn)

    
    print('总耗时：',time.strftime('%H:%M:%S',time.localtime(time.time()-d_time)))

        
        

#获得股票代码
import tushare as ts

def acq_code(ty):

    #获得全部股票代码
#    pro = ts.pro_api('b9d571514f98301a19ea719b270425aeceb086552c3f3965e49bd84c') #''的token可以到“https://tushare.pro/”注册获得
    pro = ts.pro_api('5ae1c2cea081bde0fb95295e7bfa302c5b87cc7bb967ccd5c3cee38a') #''的token可以到“https://tushare.pro/”注册获得
    pool = pro.stock_basic(exchange = '',list_status = 'L',adj = 'qfq',fields = 'ts_code')
    print('成功获取全部', len(pool), '只股票代码……')

    #筛选所需的股票代码
    stock_code = []
    for i in range(len(pool.ts_code)):
#        if i==2: #在调试时，减少数据量。正常运行时关掉该判断（该行和下一行）
#            break
        if ty == 1:
            if pool.ts_code[i][0:2]=='60': #上交所主板 
                stock_code.append(pool.ts_code[i])
        elif ty == 2: #深交所主板
            if pool.ts_code[i][0:3]=='000': #上交所主板 
                stock_code.append(pool.ts_code[i])
        elif ty == 3:
            if pool.ts_code[i][0:3]=='002': #上交所主板 
                stock_code.append(pool.ts_code[i])
        elif ty == 0:
            if (pool.ts_code[i][0:2]=='00' or pool.ts_code[i][0:2]=='60'): #只保留主板和中小板的股票
                stock_code.append(pool.ts_code[i])

    print('完成筛选，只保留主板和中小板的股票共：', len(stock_code), '只股票代码……')

    return stock_code



#根据股票代码获得股票数据
import tushare as ts
import datetime

def acq_data(stock_code, start_date, end_date):
    
    #获取股票数据
    pro = ts.pro_api('5ae1c2cea081bde0fb95295e7bfa302c5b87cc7bb967ccd5c3cee38a') #token可以到“https://tushare.pro/”注册获得
    try:
        df = pro.query('daily', ts_code = stock_code, start_date = start_date, end_date = end_date) #获取该股票历史数据
    except:
        print('连接超时，等待60秒再继续……') #普通用户，tushare每分钟只能读取200次数据。
        time.sleep(60)
        df = pro.query('daily', ts_code = stock_code, start_date = start_date, end_date = end_date) #获取该股票历史数据
    
    return df

        
    
#整理线性回归所需的数据           
import math
import numpy as np
from sklearn import preprocessing, model_selection, svm

def general_data(df, stock_code, forecast_out):
    
    forecast_col = 'close' # 定义预测列变量，它存放研究对象的标签名
#    forecast_out = 5 # 定义预测天数，这里设置为5天
    #forecast_out = int(math.ceil(0.01*len(df))) # 定义预测天数，这里设置为所有数据量长度的1%
    
    df = df[['open', 'high', 'low', 'close', 'vol']] # 只用到df中下面的几个字段

    # 构造两个新的列。模型认为这两个列的数据更能代表预测依据
    df['HL_PCT'] = (df['high'] - df['close']) / df['close'] * 100.0 # HL_PCT为股票最高价与最低价的变化百分比
    df['PCT_change'] = (df['close'] - df['open']) / df['open'] * 100.0 # HL_PCT为股票收盘价与开盘价的变化百分比
        
    df = df[['close', 'HL_PCT', 'PCT_change', 'vol']] # 这是真正用到的特征字段

    df.fillna(-99999, inplace=True) # 因为scikit-learn并不会处理空数据，需要把为空的数据都设置为一个比较难出现的值，这里取-99999，
    df['label'] = df[forecast_col].shift(-forecast_out) # 用label代表该字段，是预测结果。# 通过让与Adj. Close列的数据往前移动1%行来表示

    X = np.array(df.drop(['label'], 1)) # 最后生成真正在模型中使用的数据X和y和预测时用到的数据数据X_lately
    X = preprocessing.scale(X) # TODO 此处尚有疑问
        
#    X_lately = X[-forecast_out:] # 上面生成label列时留下的最后1%行的数据，这些行并没有label数据，因此我们可以拿他们作为预测时用到的输入数据
    X_lately = X[0:forecast_out,:] # 取数组的前面forecast_out行作为预测时用到的输入数据

    X = X[:-forecast_out]
        
    df.dropna(inplace=True) # 抛弃label列中为空的那些行
    y = np.array(df['label'])
                        
    return X, y, X_lately



#进行线性回归计算
# scikit-learn从0.2版本开始废弃cross_validation，改用model_selection
from sklearn import preprocessing, model_selection, svm
from sklearn.linear_model import LinearRegression

def general_linear(X, y, X_lately):
    
    #初始化变量
    income = 0
    accuracy = 0
    forecast_set = []
    
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2) # 把X和y分成训练和测试两部分
    
    clf = LinearRegression(n_jobs=-1) # 生成scikit-learn的线性回归对象
    
    clf.fit(X_train, y_train) # 开始训练
    accuracy = clf.score(X_test, y_test) # 用测试数据评估准确性
    forecast_set = clf.predict(X_lately) # 进行预测
        
    if len(forecast_set) > 0:
        income = forecast_set[0] - y_test[0] #股票计划在n天后卖出,这里用n天后的预测值，减去运算当天的数值，作为收益。
    
    return income, accuracy, forecast_set



#绘制图表。（只做运算时可忽略此步骤）
import matplotlib.pyplot as plt
from matplotlib import style
import datetime

def draw_plot(df, end_date, stock_code, forecast_set):
    
    style.use('ggplot') # 修改matplotlib样式
    one_day = 86400 #一天有86400秒
    
    df['Forecast'] = np.nan # 在df中新建Forecast列，用于存放预测结果的数据
    
    last_date = datetime.datetime.strptime(end_date,'%Y%m%d') # 取df最后一天的日期
    last_unix = last_date.timestamp()
    next_unix = last_unix + one_day

    # 遍历预测结果，用它往df追加行。这些行除了Forecast字段，其他都设为np.nan
    for i in forecast_set:
        next_date = datetime.datetime.fromtimestamp(next_unix)
        next_unix += one_day
        
        # [np.nan for _ in range(len(df.columns) - 1)]生成不包含Forecast字段的列表
        # 而[i]是只包含Forecast值的列表
        # 上述两个列表拼接在一起就组成了新行，按日期追加到df的下面
        df.loc[next_date] = [np.nan for _ in range(len(df.columns) - 1)] + [i]

    # 开始绘图
    df['close'].plot()
    df['Forecast'].plot()
    plt.legend(loc=4)
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.title(stock_code)
    plt.show()



#将线性回归的计算结果追加到数组
def append_data(stock_code, income, accuracy, result):

    model_income = { #定义和赋值字典。这里将保存income（收入）和accuracy（准确率）两个数据。
        'stock_code' : stock_code,
        'income' : income,
        'accuracy' : accuracy
    }
    
    result.append(model_income)
    
    return result


#数组排序后保存到文件
import pandas as pd

def save_to_file(result, fn):
    
    result = pd.DataFrame(result) #转为DataFrame格式
    
    if len(result) > 0:
        result = result.sort_values('income', ascending = False) #根据字段'income'排序

    result.to_csv(fn, index = False) #保存到文件
    
    return fn



#这是计算data_corr的函数。在计算data_linear的同时，可以顺便把相关系数也计算出来
import pandas as pd

def cal_corr(stock_code, sts_data):
    print('正在计算相关系数……')
    df = {}
    for i in range(len(stock_code)):
        df[stock_code[i]] = sts_data[i] #将股票数据存入字典   

    df_corr=pd.DataFrame(dict([(k, pd.Series(v)) for k, v in df.items()])) #字典转为dataframe格式，以便使用corr函数。由于数据不等长，需处理。
    
    cal_corr = df_corr.corr() #计算相关系数

    fn = 'data_corr.csv'
    cal_corr.to_csv(fn) #保存到文件
    
    return fn



if __name__ == '__main__': #运行程序
    main_data_linear() 

成功获取全部 3754 只股票代码……
完成筛选，只保留主板和中小板的股票共： 1493 只股票代码……


d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

...

d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/use

符合要求的股票共有： 1370 只。
完成！线性回归运行结果保存在文件： data_linear.csv
总耗时： 08:09:25


d:\python\python37\lib\site-packages\ipykernel_launcher.py:153: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
d:\python\python37\lib\site-packages\ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
